In [ ]:
import sys
print(sys.executable)



In [ ]:
!pip install PyMuPDF
!pip install pdfplumber
!pip install camelot-py[cv]




In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import os
import fitz  # PyMuPDF
import pdfplumber
import camelot
from datetime import datetime
import shutil



ImportError: Unable to import required dependencies:
numpy: cannot import name 'ModuleDeprecationWarning' from 'numpy._globals' (c:\Users\91832\Desktop\RAG_Model_development\venv_new\lib\site-packages\numpy\_globals.py)

In [ ]:
import sys
print(sys.executable)


In [ ]:

# Step 2: Define the specific file path 
pdf_path = "C:/Users/91832/Desktop/RAG_Model_development/notebook/Sample Financial Statement.pdf"

# Step 3: Process the specific PDF file
if os.path.exists(pdf_path):
    print(f"Processing file: {pdf_path}")

    # Open and process the PDF
    doc = fitz.open(pdf_path)
    print(f"Number of pages in the PDF: {len(doc)}")

    # Example: Extract text or tables from the PDF
    for page_num in range(len(doc)):
        page = doc[page_num]
        text = page.get_text()
        print(f"Page {page_num + 1} Text: {text[:10]}...")  # Display the first 200 characters of the page

    doc.close()
else:
    print(f"File not found: {pdf_path}")

In [ ]:

def parse_pdf_holistically(pdf_path, output_folder="bot_raw_data"):
    # Reset output folders if they already exist
    if os.path.exists(output_folder):
        shutil.rmtree(output_folder)
        print(f"Removed existing folder: {output_folder}")

    os.makedirs(output_folder, exist_ok=True)
    text_folder = os.path.join(output_folder, "text")
    table_folder = os.path.join(output_folder, "table")
    os.makedirs(text_folder, exist_ok=True)
    os.makedirs(table_folder, exist_ok=True)

    # Generate timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Step 1: Extract text and save to text folder
    print("Extracting text with pdfplumber...")
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            try:
                text = page.extract_text()
                if text:
                    text_file_path = os.path.join(text_folder, f"page_{i + 1}_{timestamp}.txt")
                    with open(text_file_path, "w", encoding="utf-8") as text_file:
                        text_file.write(text)
                    print(f"Page {i + 1}: Text saved to {text_file_path}")
                else:
                    print(f"Page {i + 1}: No text found.")
            except Exception as e:
                print(f"Page {i + 1}: Error extracting text: {e}")

    # Step 2: Extract tables with Camelot
    print("Extracting tables with Camelot...")
    valid_tables = 0
    skipped_pages = []

    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            page_number = i + 1
            try:
                tables = camelot.read_pdf(pdf_path, pages=str(page_number), flavor="stream")
                if len(tables) > 0:
                    for j, table in enumerate(tables):
                        table_file_path = os.path.join(
                            table_folder, f"table_page_{page_number}_table_{j + 1}_{timestamp}.csv"
                        )
                        table.df.to_csv(table_file_path, index=False)
                        print(f"Page {page_number}: Table {j + 1} saved to {table_file_path}")
                        valid_tables += 1
                else:
                    print(f"Page {page_number}: No tables detected.")
            except Exception as e:
                print(f"Page {page_number}: Error extracting table: {e}")
                skipped_pages.append(page_number)

    # Step 3: Fallback for skipped pages using pdfplumber
    print("Attempting fallback for skipped pages...")
    for page_number in skipped_pages:
        try:
            with pdfplumber.open(pdf_path) as pdf:
                page = pdf.pages[page_number - 1]
                table = page.extract_table()
                if table:
                    df = pd.DataFrame(table[1:], columns=table[0])
                    fallback_file_path = os.path.join(
                        table_folder, f"fallback_table_page_{page_number}_{timestamp}.csv"
                    )
                    df.to_csv(fallback_file_path, index=False)
                    print(f"Fallback table saved for page {page_number} to {fallback_file_path}")
        except Exception as e:
            print(f"Page {page_number}: Fallback failed with error: {e}")

    # Step 4: Log skipped pages and valid table count
    print(f"Total valid tables extracted: {valid_tables}")
    if skipped_pages:
        print(f"Skipped pages due to errors: {skipped_pages}")
        skipped_pages_file = os.path.join(output_folder, "skipped_pages.txt")
        with open(skipped_pages_file, "w") as log_file:
            log_file.write("\n".join(map(str, skipped_pages)))
        print(f"Skipped pages logged to {skipped_pages_file}")

    # Step 5: Combine all valid tables into one CSV
    print("Combining all valid tables...")
    all_csv_files = [
        os.path.join(table_folder, f) for f in os.listdir(table_folder) if f.endswith(".csv")
    ]
    if all_csv_files:
        combined_tables = pd.concat([pd.read_csv(file) for file in all_csv_files], ignore_index=True)
        combined_csv_path = os.path.join(output_folder, f"combined_tables_{timestamp}.csv")
        combined_tables.to_csv(combined_csv_path, index=False)
        print(f"All valid tables combined and saved to {combined_csv_path}")
    else:
        print("No valid tables to combine.")

    return text_folder, table_folder, skipped_pages

# Example usage
pdf_path = "C:/Users/91832/Desktop/RAG_Model_development/notebook/Sample Financial Statement.pdf"

output_folder = "bot_raw_data"  # Folder to save output
text_folder, table_folder, skipped_pages = parse_pdf_holistically(pdf_path, output_folder)


In [ ]:
import shutil

# Define the folder and zip file paths
folder_to_zip = "bot_raw_data"  # The folder to compress
zip_file_path = "bot_raw_data.zip"  # The resulting zip file

# Create the zip file
shutil.make_archive(base_name=folder_to_zip, format="zip", root_dir=folder_to_zip)

print(f"Zip file created at: {zip_file_path}")

In [ ]:
!pip install missingno


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

# Step 1: Load all table files as DataFrames
def load_table_files(table_path):
    """Load all table files as DataFrames."""
    table_files = [os.path.join(table_path, f) for f in os.listdir(table_path) if f.endswith('.csv')]
    tables = {os.path.basename(f): pd.read_csv(f) for f in table_files}
    return tables

# Step 2: Analyze missing values in table files
def analyze_missing_values(tables):
    """Generate missing value visualizations and reports for table files."""
    for name, df in tables.items():
        print(f"Missing Values for {name}:")
        print(df.isnull().sum())
        msno.matrix(df)
        plt.title(f"Missing Data Visualization: {name}")
        plt.show()

# Step 3: Generate descriptive statistics and visualizations for tables
def generate_table_statistics(tables):
    """Generate descriptive statistics and correlation heatmaps for tables."""
    for name, df in tables.items():
        print(f"Descriptive Stats for {name}:")
        print(df.describe())
        if not df.select_dtypes(include=['number']).empty:  # Check if numerical columns exist
            sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
            plt.title(f"Correlation Heatmap: {name}")
            plt.show()

# Example execution (tables only)
table_path = "/content/bot_raw_data/table"

# Step 1: Load the table data
tables = load_table_files(table_path)

# Step 2: Analyze missing values
analyze_missing_values(tables)

# Step 3: Generate descriptive statistics
generate_table_statistics(tables)


In [ ]:


# Step 1: Define the folder path containing text files
text_path = 'C:\Users\91832\Desktop\RAG_Model_development\notebook\bot_raw_data\text'

# Step 2: Combine all text files into a single dataset
combined_text = ""
page_metadata = []

for file_name in sorted(os.listdir(text_path)):
    if file_name.endswith(".txt"):
        with open(os.path.join(text_path, file_name), "r", encoding="utf-8") as f:
            page_text = f.read()
            combined_text += f"Page {file_name} Content:\n{page_text}\n\n"
            page_metadata.append({"file_name": file_name, "content": page_text})

# Step 3: Save the combined text to a file
output_file = "text_final_output.txt"
with open(output_file, "w", encoding="utf-8") as f:
    f.write(combined_text)

print(f"Combined text saved to {output_file}")

# (Optional) Check if page metadata is preserved
for page in page_metadata[:3]:  # Display the first 3 pages for verification
    print(f"File: {page['file_name']}\nContent: {page['content'][:200]}...\n")


In [ ]:
# Load combined text from text_final_output.txt
def load_combined_text(file_path):
    """Load the combined text file."""
    with open(file_path, "r", encoding="utf-8") as f:
        combined_text = f.read()
    return combined_text

def process_combined_text(combined_text):
    """Process combined text to generate word counts and word clouds."""
    word_counts = Counter(combined_text.split())

    # Generate WordCloud
    wordcloud = WordCloud(width=800, height=400).generate_from_frequencies(word_counts)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title("WordCloud: Combined Text")
    plt.show()

    return word_counts

def save_word_count_report(word_counts, output_file):
    """Save word counts to a text file."""
    with open(output_file, "w", encoding="utf-8") as f:
        for word, count in word_counts.most_common(50):
            f.write(f"{word}: {count}\n")
    print(f"Word counts saved to {output_file}")

# Main Execution
combined_text_path = "text_final_output.txt"
word_count_output = "word_count_report.txt"

# Load the combined text
combined_text = load_combined_text(combined_text_path)

# Process the combined text
word_counts = process_combined_text(combined_text)

# Save the word count report
save_word_count_report(word_counts, word_count_output)


In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm




In [ ]:
import nltk
import os

# Download NLTK resources if not already available
nltk_data_path = '/content/nltk_data'
nltk.data.path.append(nltk_data_path)

# Check if punkt and stopwords are downloaded
if not os.path.exists(f"{nltk_data_path}/tokenizers/punkt"):
    nltk.download('punkt', download_dir=nltk_data_path)
if not os.path.exists(f"{nltk_data_path}/corpora/stopwords"):
    nltk.download('stopwords', download_dir=nltk_data_path)

print("NLTK resources are ready.")



In [ ]:

text_file_path = "/content/text_final_output.txt"

# Ensure the file exists
if os.path.exists(text_file_path):
    with open(text_file_path, 'r', encoding='utf-8') as file:
        combined_text = file.read()
    print(f"Loaded text data from {text_file_path}")
else:
    print(f"File not found: {text_file_path}")



In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm')
doc = nlp(combined_text)

sentences = [sent.text for sent in doc.sents]
words = [token.text for token in doc if token.is_alpha]

print(f"Number of sentences: {len(sentences)}")
print(f"Number of words: {len(words)}")


In [ ]:
# Remove stopwords and keep only meaningful words
filtered_words = [token.text.lower() for token in doc if token.is_alpha and not token.is_stop]

print(f"Filtered words count: {len(filtered_words)}")
print(f"Sample filtered words: {filtered_words[:10]}")  # Display the first 20 filtered words


In [ ]:
print(combined_text[:500])  # View the first 500 characters of the input text


In [ ]:
import spacy
from spacy.tokenizer import Tokenizer
from spacy.util import compile_prefix_regex, compile_infix_regex, compile_suffix_regex

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

# Custom tokenizer: Define custom splitting rules
prefix_re = compile_prefix_regex(nlp.Defaults.prefixes)
infix_re = compile_infix_regex(nlp.Defaults.infixes + [r"(?<=[a-zA-Z])(?=[A-Z])"])  # Split by camel case
suffix_re = compile_suffix_regex(nlp.Defaults.suffixes)

# Assign custom tokenizer
custom_tokenizer = Tokenizer(
    nlp.vocab,
    rules=nlp.Defaults.tokenizer_exceptions,
    prefix_search=prefix_re.search,
    infix_finditer=infix_re.finditer,
    suffix_search=suffix_re.search,
)
nlp.tokenizer = custom_tokenizer

# Apply custom tokenizer
doc = nlp(combined_text)
tokens = [token.text for token in doc if token.is_alpha]

print(f"Sample tokens: {tokens[:50]}")  # Display the first 50 tokens


In [ ]:
!pip install wordninja


import wordninja

# Tokenize using Wordninja
tokens = []
for token in combined_text.split():  # Split the combined text into rough tokens first
    tokens.extend(wordninja.split(token))  # Split each rough token into meaningful words

print(f"Sample tokens: {tokens[:50]}")  # Display the first 50 tokens



In [20]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))
filtered_tokens = [word for word in tokens if word.lower() not in stop_words]


In [21]:
from collections import Counter

word_freq = Counter(filtered_tokens)
most_common_words = word_freq.most_common(20)



In [ ]:

print(f"Filtered tokens count: {len(filtered_tokens)}")
print(f"Sample filtered tokens: {filtered_tokens[:20]}")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Create a DataFrame for visualization
word_freq_df = pd.DataFrame(most_common_words, columns=['Word', 'Frequency'])

# Plot
plt.figure(figsize=(10, 6))
sns.barplot(x='Frequency', y='Word', data=word_freq_df)
plt.title("Top 20 Most Frequent Words")
plt.xlabel("Frequency")
plt.ylabel("Words")
plt.show()


In [ ]:
# Recreate sentences if needed
sentences = " ".join(tokens).split('.')  # Basic sentence split by period

# Calculate sentence lengths
sentence_lengths = [len(sent.split()) for sent in sentences]

# Plot sentence length distribution
plt.figure(figsize=(10, 6))
sns.histplot(sentence_lengths, bins=30, kde=True)
plt.title("Sentence Length Distribution")
plt.xlabel("Number of Words per Sentence")
plt.ylabel("Frequency")
plt.show()



In [ ]:
from nltk.util import ngrams
from collections import Counter

# Generate bigrams
bigrams = list(ngrams(filtered_tokens, 2))
bigram_freq = Counter(bigrams).most_common(20)

# Generate trigrams
trigrams = list(ngrams(filtered_tokens, 3))
trigram_freq = Counter(trigrams).most_common(20)

print("Top 20 Bigrams:", bigram_freq)
print("Top 20 Trigrams:", trigram_freq)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Vectorize the text
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
text_matrix = vectorizer.fit_transform(filtered_tokens)

# Fit LDA model
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(text_matrix)

# Display top words for each topic
for i, topic in enumerate(lda.components_):
    print(f"Topic {i}:")
    print([vectorizer.get_feature_names_out()[index] for index in topic.argsort()[-10:]])


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Generate word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(" ".join(filtered_tokens))

# Display word cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Word Cloud of Filtered Tokens")
plt.show()


In [ ]:
# Reconstruct sentences from WordNinja tokens
cleaned_text = " ".join(filtered_tokens)  # Use filtered tokens from WordNinja


# Save reconstructed cleaned text to a file
cleaned_text_file_path = "/content/cleaned_text.txt"

with open(cleaned_text_file_path, "w", encoding="utf-8") as file:
    file.write(cleaned_text)

print(f"Cleaned text saved to: {cleaned_text_file_path}")



In [ ]:

from textblob import TextBlob

# Analyze sentiment for the entire text
text_blob = TextBlob(cleaned_text)
print(f"Polarity: {text_blob.sentiment.polarity}")
print(f"Subjectivity: {text_blob.sentiment.subjectivity}")

In [ ]:
nlp = spacy.load('en_core_web_sm')  # Load spaCy's pre-trained model
doc = nlp(cleaned_text)

# Extract and print entities
entities = [(ent.text, ent.label_) for ent in doc.ents]
print(f"Sample Entities: {entities[:10]}")


In [31]:
# Post-process WordNinja output for cleaner input to spaCy
postprocessed_text = " ".join([word.capitalize() if word.islower() else word for word in filtered_tokens])


In [ ]:
from collections import Counter
import matplotlib.pyplot as plt

entity_counts = Counter([ent.label_ for ent in doc.ents])
plt.bar(entity_counts.keys(), entity_counts.values())
plt.title("Entity Frequency")
plt.xlabel("Entity Type")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Save entities to a CSV file
import pandas as pd

entities_df = pd.DataFrame([(ent.text, ent.label_) for ent in doc.ents], columns=["Entity", "Type"])
entities_df.to_csv("ner_entities.csv", index=False)
print("Entities saved to ner_entities.csv")


This code uses a pre-trained summarization model, facebook/bart-large-cnn, from the Hugging Face Transformers library to generate a concise summary of the input text. The model processes the first 1000 characters of the cleaned text and outputs a summary constrained by the max_length and min_length parameters. It is particularly useful for distilling long financial documents or reports into key highlights.

In [ ]:
from transformers import pipeline

# Use a pre-trained summarization model
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

summary = summarizer(cleaned_text[:1000], max_length=100, min_length=30, do_sample=False)
print("Summary:", summary[0]['summary_text'])


In [ ]:
from nltk.tokenize import sent_tokenize
chunks = [cleaned_text[i:i+1000] for i in range(0, len(cleaned_text), 1000)]

import json

# Save text chunks to a JSON file
text_chunks_file_path = "/content/cleaned_text_chunks.json"

with open(text_chunks_file_path, "w", encoding="utf-8") as file:
    json.dump(chunks, file, indent=4)

print(f"Text chunks saved to: {text_chunks_file_path}")



In [ ]:
print(f"Length of cleaned_text: {len(cleaned_text)}")


In [ ]:
# Refined chunking
chunk_size = 1000  # Size of each chunk
chunks = [cleaned_text[i:i+chunk_size] for i in range(0, len(cleaned_text), chunk_size)]

# Print chunks for debugging
for idx, chunk in enumerate(chunks):
    print(f"Chunk {idx + 1}:\n{chunk[:100]}...\n")


In [38]:
import re
headers = re.findall(r'\n[A-Z ]+\n', combined_text)


In [ ]:
print(combined_text[:1000])  # View the first 1000 characters


In [ ]:
headers = re.findall(r'\n[A-Z ]+\n', combined_text)
print(f"Headers Found: {headers}")


Avoid Duplicate headers

In [ ]:
# Remove duplicates while preserving order
unique_headers = []
for header in headers:
    if header.strip() not in unique_headers:
        unique_headers.append(header.strip())

print(f"Unique Headers: {unique_headers}")


In [ ]:
# Split the text into sections using the headers
sections = re.split(r'\n[A-Z ]+\n', combined_text)

# Map each section to its corresponding header
header_section_mapping = {}
for idx, section in enumerate(sections):
    header = unique_headers[idx] if idx < len(unique_headers) else "Unknown Section"
    header_section_mapping[header] = section.strip()

# Print each header with its corresponding content (first 100 characters)
for header, content in header_section_mapping.items():
    print(f"Header: {header}\nContent: {content[:100]}...\n")


In [ ]:
import pandas as pd

# Create a DataFrame
df = pd.DataFrame(list(header_section_mapping.items()), columns=["Header", "Content"])

# Save to CSV
df.to_csv("headers_and_sections.csv", index=False)
print("Headers and sections saved to 'headers_and_sections.csv'.")


Data Preprocessing For  Table Data

In [ ]:
def handle_missing_values(table_folder, output_folder):
    """
    Handle missing values in tables by replacing empty or null-like values with NaN.
    Parameters:
        table_folder (str): Path to the folder containing raw table files.
        output_folder (str): Path to the folder for saving cleaned tables.
    """
    # Remove old files in the output folder if it exists
    if os.path.exists(output_folder):
        shutil.rmtree(output_folder)
        print(f"Old files removed from: {output_folder}")

    # Recreate the output folder
    os.makedirs(output_folder, exist_ok=True)

    table_files = [os.path.join(table_folder, f) for f in os.listdir(table_folder) if f.endswith('.csv')]

    for table_file in table_files:
        try:
            # Load the table
            df = pd.read_csv(table_file)

            # Replace missing or inconsistent values with NaN
            df.replace(to_replace=[r'^\s*$', r'^-$', None, 'null', 'NaN'], value=np.nan, regex=True, inplace=True)

            # Log missing values count
            missing_values = df.isnull().sum().sum()
            print(f"Missing values in table {os.path.basename(table_file)}: {missing_values}")

            # Save the cleaned table to the output folder
            output_file_path = os.path.join(output_folder, os.path.basename(table_file))
            df.to_csv(output_file_path, index=False)

            print(f"Cleaned table saved: {output_file_path}")
        except Exception as e:
            print(f"Error processing table {table_file}: {e}")

# Example usage
raw_table_folder = "/content/bot_raw_data/table"  # Replace with your raw table folder path
cleaned_table_folder = "/content/cleaned_table_files"  # Replace with your desired cleaned table folder path
handle_missing_values(raw_table_folder, cleaned_table_folder)


In [ ]:
def verify_missing_values(cleaned_table_folder):
    """
    Verify missing values in cleaned tables.
    Parameters:
        cleaned_table_folder (str): Path to the folder containing cleaned table files.
    """
    table_files = [os.path.join(cleaned_table_folder, f) for f in os.listdir(cleaned_table_folder) if f.endswith('.csv')]

    for table_file in table_files:
        try:
            # Load the table
            df = pd.read_csv(table_file)

            # Check for missing values
            print(f"Missing values in table {os.path.basename(table_file)}:")
            print(df.isnull().sum())  # Column-wise missing values
            print("\n")
        except Exception as e:
            print(f"Error verifying table {table_file}: {e}")

# Example usage
verify_missing_values(cleaned_table_folder)


In [ ]:
import pandas as pd
import os
import shutil

def flatten_tables(input_folder, output_folder):
    """
    Flatten tables without modifying or separating year columns.
    Parameters:
        input_folder (str): Path to folder containing cleaned table files.
        output_folder (str): Path to save flattened tables.
    """
    # Clear the output folder if it exists
    if os.path.exists(output_folder):
        shutil.rmtree(output_folder)
        print(f"Cleared existing files in: {output_folder}")
    os.makedirs(output_folder, exist_ok=True)

    table_files = [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.endswith('.csv')]

    for table_file in table_files:
        try:
            # Load the table
            df = pd.read_csv(table_file)

            # Flatten the data: Ensure rows and columns are properly aligned
            df = df.reset_index(drop=True)

            # Save the processed table
            output_file_path = os.path.join(output_folder, os.path.basename(table_file))
            df.to_csv(output_file_path, index=False)
            print(f"Flattened table saved: {output_file_path}")
        except Exception as e:
            print(f"Error processing table {table_file}: {e}")

# Example usage
cleaned_table_folder = "/content/cleaned_table_files"  # Replace with your cleaned table folder path
flattened_table_folder = "/content/flattened_table_files"  # Replace with your desired output folder path
flatten_tables(cleaned_table_folder, flattened_table_folder)


In [ ]:


# Tokenize cleaned_text using WordNinja
cleaned_text_tokens = wordninja.split(cleaned_text)
print(f"Tokenized cleaned_text count (WordNinja): {len(cleaned_text_tokens)}")

# Compare with filtered_tokens
if len(cleaned_text_tokens) == len(filtered_tokens):
    print("No missing data: Token counts match.")
else:
    print("Mismatch detected: Check for possible issues.")


Generate embeddings for your cleaned_text_chunks.json and Flattened_table_files:

In [ ]:
# Load cleaned text chunks
with open('cleaned_text_chunks.json', 'r') as file:
    cleaned_text_chunks = json.load(file)

print(f"Loaded {len(cleaned_text_chunks)} text chunks.")

In [ ]:
#Path to flattened table files
flattened_table_folder = 'flattened_table_files'

# List all CSV files in the folder
flattened_table_files = [os.path.join(flattened_table_folder, f) for f in os.listdir(flattened_table_folder) if f.endswith('.csv')]
print(f"Found {len(flattened_table_files)} flattened table files.")

In [ ]:
pip install sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer

# Load the pre-trained Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')  # You can choose other models if required


In [ ]:
# Generate embeddings for cleaned text chunks
text_embeddings = model.encode(cleaned_text_chunks)

# Save embeddings as a NumPy array
np.save('cleaned_text_chunks_embeddings.npy', text_embeddings)
print(f"Text embeddings saved to cleaned_text_chunks_embeddings.npy.")


In [ ]:
# Path to save table embeddings
table_embeddings_folder = 'Table_Embeddings'
os.makedirs(table_embeddings_folder, exist_ok=True)

# Process each flattened table
for table_file in flattened_table_files:
    try:
        # Load the table
        df = pd.read_csv(table_file)

        # Save the table as a CSV in the embeddings folder for reference
        table_csv_file = os.path.join(table_embeddings_folder, os.path.basename(table_file))
        df.to_csv(table_csv_file, index=False)
        print(f"Saved table data to {table_csv_file}")

        # Select text columns for embedding
        text_columns = [col for col in df.columns if df[col].dtype == 'object']

        # Generate and save embeddings for each text column
        for col in text_columns:
            print(f"Generating embeddings for column '{col}' in file '{os.path.basename(table_file)}'...")
            column_embeddings = model.encode(df[col].fillna('').tolist())

            # Save the embeddings
            embedding_file = os.path.join(table_embeddings_folder, f"{os.path.basename(table_file).replace('.csv', '')}_{col}_embeddings.npy")
            np.save(embedding_file, column_embeddings)
            print(f"Saved embeddings to {embedding_file}")
    except Exception as e:
        print(f"Error processing file {table_file}: {e}")


store these embeddings in a vector database like FAISS for efficient retrieval.

In [ ]:
!pip install faiss-cpu


In [ ]:
# Path to the folder containing table embeddings
embedding_folder = "Table_Embeddings"  # Replace with the correct path

# Load text mapping
with open("cleaned_text_chunks.json", "r") as file:
    text_mapping = json.load(file)

# Enhance text mapping
text_mapping = [{"type": "text", "content": chunk} for chunk in text_mapping]

# Generate table mapping
table_mapping = []
for file in os.listdir(embedding_folder):
    if file.endswith(".csv"):
        table_name = os.path.splitext(file)[0]
        table_path = os.path.join(embedding_folder, file)

        # Load table
        table_df = pd.read_csv(table_path)

        # Add each row to the mapping
        for idx, row in table_df.iterrows():
            table_mapping.append({
                "type": "table",
                "table_name": table_name,
                "row_index": idx,
                "row_content": row.to_dict()
            })

# Combine text and table mappings
unified_mapping = text_mapping + table_mapping

# Save unified mapping
with open("enhanced_unified_mapping.json", "w") as file:
    json.dump(unified_mapping, file, indent=4)

print(f"Unified mapping saved with {len(unified_mapping)} entries.")  # Should match embeddings


In [ ]:
import numpy as np
import faiss

# Generate embeddings for the unified mapping
all_embeddings = []

for entry in unified_mapping:
    if entry["type"] == "text":
        # Encode text content
        all_embeddings.append(model.encode(entry["content"]))
    elif entry["type"] == "table":
        # Encode table row content as a string
        table_embedding = model.encode(" ".join(str(v) for v in entry["row_content"].values()))
        all_embeddings.append(table_embedding)

# Convert to NumPy array
all_embeddings = np.vstack(all_embeddings)

# Rebuild the FAISS index
embedding_dim = all_embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(all_embeddings)

print(f"Rebuilt FAISS index with {index.ntotal} vectors.")  # Should match len(unified_mapping)

# Save the FAISS index
faiss.write_index(index, "vector_index.faiss")


In [57]:
import faiss

# Get the embedding dimension from the array
embedding_dim = all_embeddings.shape[1]

# Initialize an index for L2 (Euclidean) distance
index = faiss.IndexFlatL2(embedding_dim)


In [ ]:
# Add embeddings to the FAISS index
index.add(all_embeddings)

print(f"FAISS index contains {index.ntotal} vectors.")


In [ ]:
faiss.write_index(index, "vector_index.faiss")
print("FAISS index saved to vector_index.faiss.")


In [ ]:
index = faiss.read_index("vector_index.faiss")
print(f"Loaded FAISS index with {index.ntotal} vectors.")


In [ ]:
# Example query (use an embedding of the same dimension)
query_embedding = all_embeddings[0].reshape(1, -1)

# Search for the 5 nearest neighbors
k = 5
distances, indices = index.search(query_embedding, k)

print("Nearest neighbors' indices:", indices)
print("Nearest neighbors' distances:", distances)


In [ ]:
with open("cleaned_text_chunks.json", "r") as file:
    mapping_data = json.load(file)
print(f"Mapping data loaded with {len(mapping_data)} entries.")



In [ ]:
import os

embedding_folder = "Table_Embeddings"

# List all .npy and corresponding .csv files
npy_files = [f for f in os.listdir(embedding_folder) if f.endswith(".npy")]
csv_files = [f for f in os.listdir(embedding_folder) if f.endswith(".csv")]

print("Numpy Files:", npy_files)
print("CSV Files:", csv_files)


Retrival component

In [75]:
class Retriever:
    def __init__(self, model, index, mapping_data, similarity_threshold=0.5):
        """
        Initialize the retriever.
        Args:
            model: Embedding model (e.g., SentenceTransformer).
            index: FAISS index.
            mapping_data: Unified mapping with text and table content.
            similarity_threshold: Minimum similarity score to include a result.
        """
        self.model = model
        self.index = index
        self.mapping_data = mapping_data
        self.similarity_threshold = similarity_threshold

    def retrieve(self, query, k=5):
        """
        Retrieve the top-k most relevant chunks for a given query.
        Args:
            query (str): User query in natural language.
            k (int): Number of top results to retrieve.
        Returns:
            List of tuples: [(content, distance), ...]
        """
        # Generate embedding for the query
        query_embedding = self.model.encode(query)

        # Search FAISS index
        distances, indices = self.index.search(query_embedding.reshape(1, -1), k)

        # Map indices to human-readable results
        results = []
        for idx, i in enumerate(indices[0]):
            if distances[0][idx] < self.similarity_threshold:  # Apply similarity threshold
                if 0 <= i < len(self.mapping_data):  # Ensure index is valid
                    item = self.mapping_data[i]
                    if item["type"] == "text":
                        # Handle text entries
                        results.append((f"Text: {item['content']}", distances[0][idx]))
                    elif item["type"] == "table":
                        # Handle table entries
                        table_info = f"Table: {item['table_name']}, Row {item['row_index']}"
                        row_content = f"Content: {item['row_content']}"
                        results.append((f"{table_info}\n{row_content}", distances[0][idx]))
            else:
                # Log or handle invalid indices
                print(f"Warning: Low similarity score for index {i}. Skipping.")
        return results


Testing the Retrival component

In [ ]:
# Initialize retriever with a similarity threshold
retriever = Retriever(model=model, index=index, mapping_data=unified_mapping, similarity_threshold=1)

# Retrieve results
query = "What is revenue?"
results = retriever.retrieve(query, k=5)

# Print results
for i, (content, distance) in enumerate(results):
    print(f"Result {i+1}:")
    print(content)
    print(f"Distance: {distance}\n")



In [ ]:
print(f"FAISS index vectors: {index.ntotal}")
print(f"Unified mapping entries: {len(unified_mapping)}")




In [ ]:
# Count text and table entries in unified mapping
text_count = sum(1 for entry in unified_mapping if entry["type"] == "text")
table_count = sum(1 for entry in unified_mapping if entry["type"] == "table")

print(f"Text entries: {text_count}")
print(f"Table entries: {table_count}")

# Analyze similarity scores for the top results
for i, (content, distance) in enumerate(results):
    print(f"Result {i+1}: {content}")
    print(f"Distance: {distance}")
    print()


Generator Component Using Hugging Face Transformers

In [ ]:
!pip install transformers


In [ ]:
class Generator:
    def __init__(self, model="gpt2-xl", device="cuda"):
        from transformers import pipeline
        self.generator = pipeline("text-generation", model=model, device=0 if device == "cuda" else -1)

    def generate(self, query, context, max_new_tokens=100, temperature=0.5, repetition_penalty=1.2):
        """
        Generate a response based on the query and context.
        Args:
            query (str): User's query.
            context (str): Retrieved context.
            max_new_tokens (int): Maximum number of new tokens to generate.
            temperature (float): Sampling temperature for generation.
            repetition_penalty (float): Penalty for repeated tokens.
        Returns:
            str: Generated response.
        """
        prompt = (
             f"Provide a clear and concise explanation of gross profit using the given context:\n\n"
             f"Context:\n{context}\n\n"
             f"Question:\n{query}\n\n"
             f"Answer in one sentence:"
        )

        result = self.generator(
            prompt,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            repetition_penalty=repetition_penalty,
            num_return_sequences=1,
        )
        return result[0]["generated_text"]

# Example usage
generator = Generator(model="gpt2-xl")
query = "What is revenue?"
context = "revenue, expenses"
response = generator.generate(query, context)
print(response)


In [80]:
class RAGPipeline:
    def __init__(self, retriever, generator):
        self.retriever = retriever
        self.generator = generator

    def query(self, user_query, k=5):
        # Retrieve top-k results
        retrieved_results = self.retriever.retrieve(user_query, k)
        context = "\n".join([content for content, _ in retrieved_results])

        # Generate answer using the retrieved context
        answer = self.generator.generate(user_query, context)
        return answer


In [ ]:
class RAGPipeline:
    def __init__(self, retriever, generator):
        self.retriever = retriever
        self.generator = generator

    def query(self, user_query, k=5):
        retrieved_results = self.retriever.retrieve(user_query, k)
        context = "\n".join([content for content, _ in retrieved_results])
        answer = self.generator.generate(user_query, context)
        return answer

# Initialize RAG pipeline
rag_pipeline = RAGPipeline(retriever, generator)
query = "What is revenue?"
response = rag_pipeline.query(query, k=5)

print("Query:", query)
print("Response:", response)



In [ ]:
# Initialize retriever and generator
retriever = Retriever(model=model, index=index, mapping_data=unified_mapping)
generator = Generator(model="gpt2-xl")  # Use OpenAI or Hugging Face

# Initialize RAG pipeline
rag_pipeline = RAGPipeline(retriever, generator)

# Query the pipeline
user_query = "What is revenue?"
response = rag_pipeline.query(user_query, k=5)

print("Query:", user_query)
print("Response:", response)
